In [11]:
# import sympy as sp
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import display

In [12]:
def create_properties_dict(item, caminho_tabela="Composite material properties.csv"):
    '''
    Creates a dict containing the properties of the item with the same index in the csv.
    - All values are metric units with no multipliers (Pa instead of MPa)
    '''
    # Abrindo tabelas para extrair dados
    CompositeTable = pd.DataFrame()
    CompositeTable = pd.read_csv(caminho_tabela,sep=';')

    
    # # check if item is a string
    # if type(item) == str:
    #     # find the number of the row of the item in the column 'Material'
    #     item = CompositeTable[CompositeTable['Material'] == item].index[0]

    v23 = CompositeTable.loc[item,"nu_23"] # [Adimensional]
    E2 = CompositeTable.loc[item,"E_2 [Gpa]"] # [GPa]
    G23 = E2 / (2*(1+v23))

    properties_dict = {
        'E1':CompositeTable.iloc[item]["E_1 [Gpa]"],  # [GPa]
        'E2': E2, # CompositeTable.iloc[item]["E_2 [Gpa]"], # [GPa]
        'v12':CompositeTable.iloc[item]["nu_12"], # [Adimensional]
        'v21':v23,
        'v23': v23, # CompositeTable.iloc[item]["nu_23"], # [Adimensional]
        'G12':CompositeTable.iloc[item]["G_12 [Gpa]"],  # [GPa]
        'G13':CompositeTable.iloc[item]["G_12 [Gpa]"],  # [GPa]
        'G23': G23, # [GPa]
        'F1t':None,
        'F1c':None,
        'F2t':None,
        'F2c':None,
        'F6':None,
        'Material name':CompositeTable.iloc[item]["Material"],
        'Material type':CompositeTable.iloc[item]["Material type"],
        'Density per area':None,
        'Cost per area':None
        }
    
    # properties_dict  = {'Mechanical':MECHANICAL,'Strenght':STRENGH,'Physical':PHYSICAL}

    return properties_dict




In [13]:
# Definindo as funções básicas para os cálculos de compósitos

def compute_A(theta, unit = 'graus'):
    '''
    PURPOSE
    - Compute the matrix a whitch Relates laminate xy coordinates to 12
    - It is eq.5.40 on the Barbero book
    
    INPUTS
    - theta         : angle between laminate and lamina

    OUTPUTS
    - T : Transformation Matrix a (eq.5.26)
    '''
    if unit == 'graus':
        theta = np.deg2rad(theta)

    m = np.cos(theta)
    n = np.sin(theta)

    A = np.array([
        [m,n],
        [-n,m]
    ])

    return A


def compute_T(theta, unit = 'graus'):
    '''
    PURPOSE
    - Compute the matrix T whitch Relates laminate deformation to the lamina
      deformation
    - It is eq.5.40 on the Barbero book
    - Deformation_x = T * Deformation_1
    - Stress_x = T^-1 * Stress_1
    
    INPUTS
    - theta : angle between laminate and lamina in degrees
        OUTPUTS
    - T     : Transformation Matrix T (eq.5.40)'''

    if unit == 'graus':
        theta = np.deg2rad(theta)

    m = np.cos(theta)
    n = np.sin(theta)

    T = np.array([
        [m**2  ,  n**2 , 2*m*n],
        [n**2  ,  m**2 , -2*m*n],
        [-m*n , m*n  , m**2 - n**2]
    ])

    return T

def compute_Q(properties_dict):
    '''PURPOSE
    - Compute the matrix Q whitch is the inverse of the compliance matrix
    - It is eq.5.21 on the Barbero book
    - The matrix S is Q^-1

    INPUTS
    - E1         : Youngs modulus on 1 direction
    - E2         : Youngs modulus on 2 direction
    - poison12   : Poison Ration on the 12 direction 
    - G12        : Shear modulus 1 direction
    OUTPUTS
    - Q : Matrix Q (eq.5.21)'''
            
    delta = (
        1 - (properties_dict['v12']**2) * properties_dict['E2']/
        properties_dict['E1']
    )
    Q = np.zeros((3,3))
    Q[0,0] = properties_dict['E1'] / delta
    Q[1,1] = properties_dict['E2'] / delta
    Q[2,2] = properties_dict['G12']
    Q[0,1] = properties_dict['v12'] * properties_dict['E2'] / delta
    Q[1,0] = Q[0,1]
    
    return Q


def compute_QI(properties_dict):
    '''      
    PURPOSE
    - Compute the matrix QI whitch is the inverse of the inatralaminar
      compliance matrix
    - It is eq.5.22 on the Barbero book
    - The matrix SI is QI^-1
    - Sigma 4 e 5 = QI * gama 4 e 5

    INPUTS
    - G13        : Intralaminar shear modulus on 13 plane
    - E2         : Youngs modulus on 2 direction
    - poison23   : Poison Ration on the 12 direction
    OUTPUTS
    - Q* : Matrix Q* (eq.5.21)
    '''

    QI = np.zeros((2,2))
    QI[0,0] = properties_dict['G23']
    QI[1,1] = properties_dict['G13']

    return QI

In [14]:
# Criando classes para armazenar as propriedades dos laminados e lâminas
reuter_matrix = np.array([
    [1,0,0],
    [0,1,0],
    [0,0,2]
])

class Ply:
    def __init__(self,material,orientation):
        
        self.material_id = material
        self.orientaton = orientation
        self.units = {'Orientation unit':'graus'}
        
        self.properties = create_properties_dict(self.material_id)
        self.T_matrix = compute_T(self.orientaton,self.units['Orientation unit'])
        self.A_matrix = compute_A(self.orientaton,self.units['Orientation unit'])

        self.Q = compute_Q(self.properties)
        self.QI = compute_QI(self.properties)
        # QT(:,:,i)  = inv(T(:,:,i))*Q(:,:,i)*inv(T(:,:,i))'
        # self.QT = np.dot(
        #     np.dot(np.linalg.inv(self.T_matrix),self.Q),
        #     np.linalg.inv(self.T_matrix).T
        # )
        self.QT = np.dot(
            np.dot(compute_T(-orientation),self.Q ),
            compute_T(-orientation).T
            )
        self.QIT = np.dot(
            np.dot(self.A_matrix,self.QI),
            self.A_matrix.T
        )

        def get_transformed_properties(self):
            return None


class Lamina(Ply):
    def __init__(self,material,orientation,thickness,lamina_coordinate):
        self.Zk = lamina_coordinate # Coordenada do centro da lâmina em realção ao plano central do lamminado
        self.thickness = thickness # Espessura da lâmina
        if self.Zk >= 0:
            self.Zn = self.Zk + (self.thickness)/2 # Possição do topo da lamina para calcular deformação máxima
        else:
            self.Zn = self.Zk - (self.thickness)/2

        Ply.__init__(self,material,orientation)

        # Partial Extensional stiffness matrix A0:
        self.A0 = self.QT * self.thickness
        # Partial Bending extension coupling stiffness matrix B:
        self.B0 = -1*self.QT * self.thickness * self.Zk
        # Partial Bending stiffness matrix D:
        self.D0 = self.QT*(self.thickness*self.Zk**2 + (self.thickness**3)/12)
        # Partial Transverse shear stiffness:
        self.H0 = -(5/4) * self.QIT * (self.thickness - (4/self.thickness**2)
                                       * (self.thickness*self.Zk**2 + (self.thickness**3)/12))

    def get_A0(self):
        return self.A0
    def get_B0(self):
        return self.B0
    def get_D0(self):
        return self.D0
    def get_H0(self):
        return self.H0
    
    def compute_deformation_and_stress(self,laminate_deformation): 
        # Cuidado deve ser tomado em laminados de lâmina única 
        # pois estou considerando zk como sendo o meio da lamina
        self.deformation = laminate_deformation[:3] - self.Zn*laminate_deformation[3:] # epsilon_x, epsilon_y, gamma_xy
        self.stress = np.dot(np.linalg.inv(self.Q), self.deformation) # sigma_x, sigma_y, gamma_xy
        pass

    def print_lamina_deformation(self):
        print("Deformation:")
        print(self.deformation)
        print("Estresse:")
        print(self.stress)
        

class Laminate:
    def __init__(self,laminate_list):
        self.laminate_stack = []
        self.t = 0
        self.create_laminate_list_with_coordinates(laminate_list)

        for lamina in self.laminate_list:
            self.laminate_stack.append(Lamina(lamina[0],lamina[1],lamina[2],lamina[3]))

        self.compute_ABD()

    def create_laminate_list_with_coordinates(self, laminate_list):
        # Add the coordinate to the laminate matrix
        L = laminate_list
        tt = sum(i[2] for i in L)
        for i in range(len(L)):
            if i == 0:
                L[i].append(-((tt/2) - L[i][2]/2))
            else:
                L[i].append(L[i-1][3] + (L[i-1][2] + L[i][2]) / 2)
        self.laminate_list=L

    def compute_ABD(self):
        self.A = np.zeros((3,3))
        self.B = np.zeros((3,3))
        self.D = np.zeros((3,3))
        self.H = np.zeros((2,2))

        for lamina in self.laminate_stack:
            self.A += lamina.get_A0()
            self.B += lamina.get_B0()
            self.D += lamina.get_D0()
            self.H += lamina.get_H0()

        self.ABD = np.zeros((6,6))

        self.ABD[0:3,0:3] = self.A
        self.ABD[3:6,3:6] = self.D
        self.ABD[3:6,0:3] = self.B
        self.ABD[3:6,0:3] = self.B

        self.abd = np.linalg.inv(self.ABD)

    def aply_load(self, load_vector):
        self.load_vector = load_vector
        self.compute_deformation()

    
    def aply_deformation(self, deformation_vector):
        self.deformation_vector = deformation_vector
        self.compute_loading()
        
        for i in range(len(self.laminate_stack)):
            self.laminate_stack[i].compute_deformation_and_stress(self.deformation_vector)

    def compute_deformation(self):
        # h = np.linalg.inv(self.H)
        # self.lamiate_transversal_deformations = 
        abd = np.linalg.inv(self.ABD)
        self.deformation_vector = np.dot(abd,self.load_vector)

        for i in range(len(self.laminate_stack)):
            self.laminate_stack[i].compute_deformation_and_stress(self.deformation_vector)
    
    def compute_loading(self):
        self.load_vector = np.dot(self.ABD,self.load_vector)

    def print_load_vector(self):
        print('Load vector:')
        print(' N_x:',self.load_vector[0])
        print(' N_y:',self.load_vector[1])
        print('N_xy:',self.load_vector[2])
        print(' M_x:',self.load_vector[3])
        print(' M_y:',self.load_vector[4])
        print('M_xy:',self.load_vector[5])

    def print_deformation_vector(self):
        print('Deformation vector:')
        print('      u_x:',self.deformation_vector[0])
        print('      u_y:',self.deformation_vector[1])
        print(' gamma_xy:',self.deformation_vector[2])
        print('      k_x:',self.deformation_vector[3])
        print('      k_y:',self.deformation_vector[4])
        print('     k_xy:',self.deformation_vector[5])



        





# Exemplo 6.1 do Barbeiro

In [15]:
# Exemplo 6.1 do Barbeiro
mt = 28
t = 0.635 # mm
theta = 55
L = [
    [mt,+theta,t],
    [mt,-theta,t],
]

laminate = Laminate(L)
# np.set_printoptions(precision=3)

print('Exemplo 6.1 do Barbeiro:')

print('\n')
print('INPLANE STIFFNESS:')

print("\nPly 0 QT:")
display(laminate.laminate_stack[0].QT)

print("\nA:")
display(laminate.A)

print("\nB:")
display(laminate.B)

print("\nD:")
display(laminate.D)

print('\n')
print('OUT OF PLANE STIFFNESS:')

print("\nPly 0 QI:")
display(laminate.laminate_stack[0].QI)

print("\nPly 0 QIT:")
display(laminate.laminate_stack[0].QIT)


print("\nH:")
display(laminate.H)

# print("\nABD:\n")
# display(laminate.ABD)

# print("\nabd:\n")
# display(laminate.abd)

Exemplo 6.1 do Barbeiro:


INPLANE STIFFNESS:

Ply 0 QT:


array([[12.4016496 ,  5.70964889,  1.21713152],
       [ 5.70964889, 15.47166068,  3.00026153],
       [ 1.21713152,  3.00026153,  6.23855472]])


A:


array([[15.75009499,  7.25125409,  0.        ],
       [ 7.25125409, 19.64900906,  0.        ],
       [ 0.        ,  0.        ,  7.92296449]])


B:


...

array([[4.11155235, 0.        ],
       [0.        , 3.789     ]])


Ply 0 QIT:


array([[ 3.89511647, -0.15155003],
       [-0.15155003,  4.00543587]])


H:


array([[2.0611658 , 0.        ],
       [0.        , 2.11954315]])

In [16]:
# Exemplo 6.2 do Barbeiro
mt = 28
t = 0.635 # mm
theta = 0
L = [
    [mt,+theta,t]
]

print("\nLaminate list:")
laminate = Laminate(L)

print(laminate.laminate_list)

print("\nXXXX:")
# display(laminate.laminate_stack[0].thickness)

print(len(laminate.laminate_stack))


print('='*50)

print("\nPly 0 QT:")
print(laminate.laminate_stack[0].QT)

print("\nD:")
print(laminate.D)

print("\nd:")
display(np.linalg.inv(laminate.D))

# print("\nABD:")
# print(laminate.ABD)

# print("\nabd:")
abd = np.linalg.inv(laminate.ABD)
# print(abd)





print('\n'*3)

load_vector = np.array([0,0,0,1,0,0]) # [N_x, N_y, N_xy, M_x, M_y, M_xy] 
laminate.aply_load(load_vector.T)
laminate.print_deformation_vector()

print('~'*50)

print(np.dot(abd,load_vector))


Laminate list:
[[28, 0, 0.635, -0.0]]

XXXX:
1

Ply 0 QT:
[[20.8742658   3.26009417  0.        ]
 [ 3.26009417 11.89815391  0.        ]
 [ 0.          0.          3.789     ]]

D:
[[0.44540095 0.06956168 0.        ]
 [0.06956168 0.25387475 0.        ]
 [0.         0.         0.08084712]]

d:


array([[ 2.34554004, -0.64267797,  0.        ],
       [-0.64267797,  4.11504394,  0.        ],
       [ 0.        ,  0.        , 12.36902493]])





Deformation vector:
      u_x: 0.0
      u_y: 0.0
 gamma_xy: 0.0
      k_x: 2.345540036272178
      k_y: -0.6426779699385768
     k_xy: 0.0
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
[ 0.          0.          0.          2.34554004 -0.64267797  0.        ]


In [17]:
Zk = 0
Q = laminate.laminate_stack[0].Q

D0 = Q*(t*Zk**2 + (t**3)/12)

display(D0)


laminate.laminate_stack[0].print_lamina_deformation()

array([[0.44540095, 0.06956168, 0.        ],
       [0.06956168, 0.25387475, 0.        ],
       [0.        , 0.        , 0.08084712]])

Deformation:
[-0.74470896  0.20405026  0.        ]
Estresse:
[-0.040069    0.02812865  0.        ]


---
# Calculos de deformação do vaso de pressão

In [18]:
R_c = 60 # mm
r_pf = 21 # mm
b_ef = 42 # mm
b_er = 42 # mm
l_c = 88 # mm

alpha_c = np.asin(r_pf/R_c)

print("Alpha:")
print(alpha_c, "deg")
print(np.rad2deg(alpha_c), "deg")

alpha_c_deg = np.rad2deg(alpha_c)

# Criando o laminado
mt = 29
t_angleply = 0.125 #(1/4) # mm #* 10**-3 # m
t_hoop = t_angleply # 0.125 # mm

L = [
    [mt,+alpha_c_deg,t_angleply],
    [mt,-alpha_c_deg,t_angleply],
    [mt,+alpha_c_deg,t_angleply],
    [mt,-alpha_c_deg,t_angleply],
    [mt,+alpha_c_deg,t_angleply],
    [mt,-alpha_c_deg,t_angleply],
    [mt,90,t_hoop],
]

laminate = Laminate(L)

pressure = 4e-3 # GPa # 4e6 # Pa

# N_x = (pressure * np.pi * R_c**2)/(2 * np.pi * R_c) # N/m
N_x = (pressure * R_c)/(2) # [10e3 Nm] tensão longitudinal
N_y = pressure * R_c # [10e3 Nm]

load_vector = np.array([N_x,N_x,0,0,0,0]) # [N_x, N_y, N_xy, M_x, M_y, M_xy]

laminate.aply_load(load_vector)

print()
laminate.print_deformation_vector()

# print('\nSurface deformations:')
# laminate.laminate_stack[-1].print_lamina_deformation()



Alpha:
0.35757110364551026 deg
20.487315114722662 deg

Deformation vector:
      u_x: 0.0007753351743533289
      u_y: 0.004398335295963933
 gamma_xy: -3.0847150750516914e-19
      k_x: -0.002730545626650036
      k_y: 0.0100874188769729
     k_xy: -0.0033391410774119452


In [19]:

exp_ex = 0.00103
exp_ey = 0.00567

for t in [0.105,0.12,0.125,0.15,0.18]:
    print('='*50)
    print('Espessura =', t)
    print()

    for mt in [29,30,31,32]:
        L = [
        [mt,+alpha_c_deg,t],
        [mt,-alpha_c_deg,t],
        [mt,+alpha_c_deg,t],
        [mt,-alpha_c_deg,t],
        [mt,+alpha_c_deg,t],
        [mt,-alpha_c_deg,t],
        [mt,90,t],
        ]

        laminate = Laminate(L)

        pressure = 4e-3 # GPa # 4e6 # Pa

        # N_x = (pressure * np.pi * R_c**2)/(2 * np.pi * R_c) # N/m
        N_x = (pressure * R_c)/(2) # [10e3 Nm] tensão longitudinal
        N_y = pressure * R_c # [10e3 Nm]

        load_vector = np.array([N_x,N_x,0,0,0,0]) # [N_x, N_y, N_xy, M_x, M_y, M_xy]

        laminate.aply_load(load_vector)

        # print('Material = ', mt)

        ex = laminate.deformation_vector[0]
        ey = laminate.deformation_vector[1]
        

        print('E1 =',laminate.laminate_stack[0].properties['E1'])
        print('deformações ex,ey:')
        print(ex,ey)

        print('Erros em relação a média de ex,ey [%]:')
        print(
            100*((exp_ex-ex)/exp_ex),
            ';',
            100*((exp_ey-ey)/exp_ey)
            )
        print()

Espessura = 0.105

E1 = 146.94
deformações ex,ey:
0.0009230180647063441 0.0052361134475761096
Erros em relação a média de ex,ey [%]:
10.386595659578255 ; 7.652320148569491

E1 = 130.0
deformações ex,ey:
0.0010608203755466505 0.005755849272063529
Erros em relação a média de ex,ey [%]:
-2.9922694705485817 ; -1.5140965090569578

E1 = 120.0
deformações ex,ey:
0.0011620263445627993 0.006114183360649439
Erros em relação a média de ex,ey [%]:
-12.81809170512614 ; -7.833921704575644

E1 = 110.0
deformações ex,ey:
0.0012831656186197433 0.006520182018415283
Erros em relação a média de ex,ey [%]:
-24.579186273761472 ; -14.994391859176082

Espessura = 0.12

E1 = 146.94
deformações ex,ey:
0.0008076408066180508 0.004581599266629098
Erros em relação a média de ex,ey [%]:
21.588271202131 ; 19.195780129998273

E1 = 130.0
deformações ex,ey:
0.0009282178286033192 0.005036368113055588
Erros em relação a média de ex,ey [%]:
9.881764213269992 ; 11.175165554575168

E1 = 120.0
deformações ex,ey:
0.00101677305

In [20]:

exp_ex = 0.00103
exp_ey = 0.00567

t_angleply = 0.333/2
t_hoop = 0.25

for mt in [29,30,31,32]:


    L = [
    [mt,+alpha_c_deg,t_angleply],
    [mt,-alpha_c_deg,t_angleply],
    [mt,+alpha_c_deg,t_angleply],
    [mt,-alpha_c_deg,t_angleply],
    [mt,+alpha_c_deg,t_angleply],
    [mt,-alpha_c_deg,t_angleply],
    [mt,90,t_hoop],
    ]

    laminate = Laminate(L)

    print('E1 =',laminate.laminate_stack[0].properties['E1'])

    pressure = 4e-3 # GPa # 4e6 # Pa

    # N_x = (pressure * np.pi * R_c**2)/(2 * np.pi * R_c) # N/m
    N_x = (pressure * R_c)/(2) # [10e3 Nm] tensão longitudinal
    N_y = pressure * R_c # [10e3 Nm]

    load_vector = np.array([N_x,N_x,0,0,0,0]) # [N_x, N_y, N_xy, M_x, M_y, M_xy]

    laminate.aply_load(load_vector)

    # print('Material = ', mt)

    ex = laminate.deformation_vector[0]
    ey = laminate.deformation_vector[1]

    print('deformações ex,ey:')
    print(ex,ey)

    print('Erros em relação a média de ex,ey [%]:')
    print(
        100*((exp_ex-ex)/exp_ex),
        ';',
        100*((exp_ey-ey)/exp_ey)
        )
    print()

E1 = 146.94
deformações ex,ey:
0.0007015378997418826 0.00237321158924572
Erros em relação a média de ex,ey [%]:
31.889524296904607 ; 58.1444164154194

E1 = 130.0
deformações ex,ey:
0.0007960151672448282 0.002629260362517543
Erros em relação a média de ex,ey [%]:
22.71697405390018 ; 53.62856503496397

E1 = 120.0
deformações ex,ey:
0.0008645393933969291 0.002808124797686552
Erros em relação a média de ex,ey [%]:
16.06413656340495 ; 50.47398945879097

E1 = 110.0
deformações ex,ey:
0.0009457257831935795 0.003013118038853349
Erros em relação a média de ex,ey [%]:
8.181962796739867 ; 46.858588380011476

